In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix

In [2]:
def eval_metrics(tn, fp, fn, tp):
    total = tn + fp + fn + tp
    accuracy = (tn + tp)/total
    precision = tp/(tp+fp)
    recall = tp/(tp + fn)
    return (accuracy, precision, recall)

# 01 - Load Data

In [3]:
# data source: Kaggle - https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
df_original = pd.read_csv('IMDB Dataset.csv')
# Sample 1K each positive and negative reviews
df = df_original.groupby('sentiment').sample(n=50, random_state=1).reset_index(drop=True)
df['sentiment'].value_counts()

sentiment
negative    50
positive    50
Name: count, dtype: int64

## 02 - Sample Sentiment Data

In [4]:
# Input string list
s1 = "The food was great!"
s2 = "This is the worst movie I ever saw"
s3 = "This food is no less than many good restaurants in town"
s4 = "I don't understand why people rated this as a good movie. There is nothing worth talking about"
strlst = [s1,s2, s3,s4]

In [5]:
# TextBlob
!pip install textblob

In [6]:
from textblob import TextBlob

for i in range(len(strlst)):
  tst = TextBlob(strlst[i])
  print(tst, ">>", tst.sentiment)

The food was great! >> Sentiment(polarity=1.0, subjectivity=0.75)
This is the worst movie I ever saw >> Sentiment(polarity=-1.0, subjectivity=1.0)
This food is no less than many good restaurants in town >> Sentiment(polarity=0.42777777777777776, subjectivity=0.3888888888888889)
I don't understand why people rated this as a good movie. There is nothing worth talking about >> Sentiment(polarity=0.5, subjectivity=0.35000000000000003)


# 03 - Evaluate Sentiment using TextBlob and return evaluation metrics

In [7]:
# Extract Sentiment and Polarity for reviews
df['Polarity'] = df['review'].apply(lambda x: TextBlob(x).sentiment.polarity)

In [8]:
df['Polarity'].describe()
meanscore = df['Polarity'].mean()

In [9]:
df['predicted_sentiment'] = df['Polarity'].apply(lambda x: 'positive' if x > meanscore else 'negative')

In [10]:
df.head()

,review,sentiment,Polarity,predicted_sentiment
0,I recently viewed Manufactured Landscapes at t...,negative,0.323333,positive
1,I figured that any horror film with Orson Well...,negative,0.001535,negative
2,Run away from this movie. Even by B-movie stan...,negative,-0.048963,negative
3,Oh dear. I was so disappointed that this movie...,negative,0.020536,negative
4,Below average blaxpoitation action / melodrama...,negative,0.248276,positive


In [11]:
cfmt = confusion_matrix(df['sentiment'], df['predicted_sentiment'], labels=['positive', 'negative'])
cfmt

array([[33, 17],
       [15, 35]], dtype=int64)

In [12]:
cfmt = confusion_matrix(df['sentiment'], df['predicted_sentiment'], labels=['negative', 'positive'])
cfmt

array([[35, 15],
       [17, 33]], dtype=int64)

In [13]:
tn, fp, fn, tp = cfmt.ravel()
(tn, fp, fn, tp)

(35, 15, 17, 33)

# 04 -Evaluate sentiment with different models

## 04 - 01 - Text Blob

In [14]:
def textblob_sentiment(df_inp):
    print ('text blob sentiment analyzer')
    df = df_inp.copy()
    df['textblob_polarity'] = df['review'].apply(lambda x: TextBlob(x).sentiment.polarity)
    meanscore = df['textblob_polarity'].mean()
    df['predicted_sentiment'] = df['textblob_polarity'].apply(lambda x: 'positive' if x > meanscore else 'negative')
    cfmt = confusion_matrix(df['sentiment'], df['predicted_sentiment'], labels=['negative', 'positive'])
    tn, fp, fn, tp = cfmt.ravel()
    acc, pre, rec = eval_metrics(tn, fp, fn, tp)
    print ('accuracy, precision and recall are', acc, pre, rec)
    return 

textblob_sentiment(df)

text blob sentiment analyzer
accuracy, precision and recall are 0.68 0.6875 0.66


# 04 - 02 - Vader

In [15]:
!pip install vaderSentiment

In [16]:

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

def vader_S(text):
    vs = analyzer.polarity_scores(text)
    if (vs['neg'] > vs['pos']):
        return 'negative'
    else:
        return 'positive'  

def vader_sentiment(df_inp):
    print ('vader sentiment analyzer')
    df = df_inp.copy()
    df['vader_sentiment'] = df['review'].apply(lambda x: vader_S(x))
    cfmt = confusion_matrix(df['sentiment'], df['vader_sentiment'], labels=['negative', 'positive'])
    tn, fp, fn, tp = cfmt.ravel()
    acc, pre, rec = eval_metrics(tn, fp, fn, tp)
    print ('accuracy, precision and recall are', acc, pre, rec)
    return 

vader_sentiment(df)

vader sentiment analyzer
accuracy, precision and recall are 0.65 0.6027397260273972 0.88


## 04 - 03 - Sentence Transformer

In [17]:
#pip install transformers
!pip install transformers
from transformers import pipeline

In [19]:
pip install --upgrade transformers

In [25]:
# Create the sentiment analysis pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
sentiment_pipeline(strlst[0])

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


RuntimeError: Failed to import transformers.models.distilbert.modeling_tf_distilbert because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [24]:
#pip install transformers
def trf_s(x):
    try:    
        y = sentiment_pipeline(x)
        #print (y)
        return (y[0]['label'])
    except:
        return 'none'


def transformer_sentiment(df_inp):
    df = df_inp.copy()
    df['transformer_sentiment'] = df['review'].apply(lambda x: trf_s(x).lower())
    cfmt = confusion_matrix(df['sentiment'], df['transformer_sentiment'], labels=['negative', 'positive'])
    tn, fp, fn, tp = cfmt.ravel()
    acc, pre, rec = eval_metrics(tn, fp, fn, tp)
    print ('accuracy, precision and recall are', acc, pre, rec)
    return 

transformer_sentiment(df)

accuracy, precision and recall are nan nan nan


C:\Users\DELL\AppData\Local\Temp\ipykernel_25052\1234202632.py:3: RuntimeWarning: invalid value encountered in scalar divide
  accuracy = (tn + tp)/total
C:\Users\DELL\AppData\Local\Temp\ipykernel_25052\1234202632.py:4: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp/(tp+fp)
C:\Users\DELL\AppData\Local\Temp\ipykernel_25052\1234202632.py:5: RuntimeWarning: invalid value encountered in scalar divide
  recall = tp/(tp + fn)
